In [46]:
import yaml
import pandas as pd

from datetime import datetime, timedelta, date
import yfinance as yf

In [47]:
with open("failed_tickers.yaml", "r") as f:
    failed_tickers = yaml.safe_load(f)

In [48]:
failed_tickers

['PJX.SI', 'M12.SI', 'N33.SI', 'XZB.SI']

In [49]:
df_stock = pd.read_csv("Stock Screener Data 31 Aug 2-16-26.csv.csv")

df_stock["Yahoo_Code"] = df_stock["Code"] + ".SI"

list_stock = []

for index, row in df_stock.iterrows():
    list_stock.append(row["Yahoo_Code"])

final_list = [item for item in list_stock if item not in failed_tickers]

In [50]:
response = yf.download(["D05.SI"], period='5d')
print(response.T)

C:\Users\john\AppData\Local\Temp\ipykernel_335708\3919928503.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  response = yf.download(["D05.SI"], period='5d')
[*********************100%***********************]  1 of 1 completed

Date             2025-08-27    2025-08-28    2025-08-29    2025-09-01  \
Price  Ticker                                                           
Close  D05.SI  5.017000e+01  5.033000e+01  5.052000e+01  5.020000e+01   
High   D05.SI  5.022000e+01  5.045000e+01  5.070000e+01  5.064000e+01   
Low    D05.SI  4.970000e+01  5.024000e+01  5.028000e+01  5.012000e+01   
Open   D05.SI  4.970000e+01  5.031000e+01  5.070000e+01  5.064000e+01   
Volume D05.SI  2.168713e+06  3.233400e+06  3.078100e+06  1.787500e+06   

Date             2025-09-02  
Price  Ticker                
Close  D05.SI  5.071000e+01  
High   D05.SI  5.074000e+01  
Low    D05.SI  5.020000e+01  
Open   D05.SI  5.020000e+01  
Volume D05.SI  2.533500e+06  


In [51]:
# response = yf.download(list_stock, period='5d
response = yf.download(final_list, period='5d')
print(response.T)

C:\Users\john\AppData\Local\Temp\ipykernel_335708\2301159167.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  response = yf.download(list_stock, period='5d')
[*********************100%***********************]  613 of 613 completed

4 Failed downloads:
['M12.SI', 'PJX.SI', 'N33.SI']: YFPricesMissingError('possibly delisted; no price data found  (period=5d)')
['XZB.SI']: YFPricesMissingError('possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')


Date                2025-08-27    2025-08-28    2025-08-29   2025-09-01  \
Price     Ticker                                                          
Adj Close M12.SI           NaN           NaN           NaN          NaN   
          N33.SI           NaN           NaN           NaN          NaN   
          PJX.SI           NaN           NaN           NaN          NaN   
          XZB.SI           NaN           NaN           NaN          NaN   
Close     1A0.SI  4.100000e-02  4.300000e-02  4.200000e-02         0.04   
...                        ...           ...           ...          ...   
Volume    Z74.SI  2.402387e+07  2.549660e+07  1.605460e+07  10042100.00   
          Z77.SI  1.680900e+05  1.224000e+05  1.000000e+05    110240.00   
          ZB9.SI  3.720000e+04           NaN           NaN          NaN   
          ZKX.SI  4.350000e+04  1.353000e+05  8.440000e+04     59500.00   
          ZXY.SI  5.930000e+04  0.000000e+00  2.054000e+05    264700.00   

Date                2025

In [53]:
failed_tickers = ['PJX.SI', 'M12.SI', 'N33.SI', 'XZB.SI']

In [54]:
with open("failed_tickers.yaml", "w") as f:
    yaml.dump(failed_tickers, f, default_flow_style=False, sort_keys=False)

In [55]:
df = response.T

In [56]:
cols = list(df.T["Close"].columns)

In [57]:
def last3_close_gte_open(open_series: pd.Series, close_series: pd.Series) -> bool:
    """
    Returns True if the last 3 days' closes are greater than opens.
    """
    diff = close_series - open_series
    last3 = diff.tail(3)
    check_diff = (last3 >= 0).all()
    return check_diff

In [58]:
def last3_gte_prev(series: pd.Series) -> bool:
    last3 = series.tail(3)  # need 4 values to compare 3 consecutive increases
    return (last3.diff().tail(2) > 0).all()

In [59]:
# cols = ['OXMU.SI']
cols = list(df.T["Close"].columns)
c_list = []
for item in cols:
    # print(item)
    close_series = df.T["Close"][item]
    open_series = df.T["Open"][item]

    check1 = last3_close_gte_open(open_series, close_series)
    check2 = last3_gte_prev(open_series)
    check3 = last3_gte_prev(close_series)

    if check1 and check2 and check3:
        c_list.append(item)

print(len(c_list))

20


In [60]:
c_list

['5GD.SI',
 '5TP.SI',
 'A04.SI',
 'AP4.SI',
 'BEZ.SI',
 'CYW.SI',
 'DHLU.SI',
 'ER0.SI',
 'H02.SI',
 'HMN.SI',
 'J85.SI',
 'S35.SI',
 'S41.SI',
 'S61.SI',
 'T14.SI',
 'TQ5.SI',
 'U96.SI',
 'Z74.SI',
 'Z77.SI',
 'ZKX.SI']

In [61]:
def score_stock(data: dict) -> dict:
    # Extract metrics
    metrics = {
        "forward_pe": data.get("forwardPE"),
        "trailing_pe": data.get("trailingPE"),
        "eps": data.get("trailingEps"),
        "ocf": data.get("operatingCashflow"),
        "shares": data.get("sharesOutstanding"),
        "cash_per_share": data.get("totalCashPerShare"),
        "current_price": data.get("currentPrice"),
        "div_yield": data.get("dividendYield"),
        "payout": data.get("payoutRatio"),
        "curr_ratio": data.get("currentRatio"),
        "quick_ratio": data.get("quickRatio"),
        "roe": data.get("returnOnEquity"),
        "roa": data.get("returnOnAssets"),
        "debt_eq": data.get("debtToEquity"),
        "rev_growth": data.get("revenueGrowth"),
        "earnings_growth": data.get("earningsGrowth"),
        "profit_margin": data.get("profitMargins")
    }

    # Compute cash flow per share
    if metrics["ocf"] and metrics["shares"]:
        metrics["cfps"] = metrics["ocf"] / metrics["shares"]
    else:
        metrics["cfps"] = None

    verdict = []
    score = 50  # neutral starting point

    # Valuation adjustments
    if metrics["forward_pe"] and metrics["forward_pe"] < 15:
        verdict.append("Cheap valuation"); score += 10
    if metrics["trailing_pe"] and metrics["trailing_pe"] != "Infinity"  and metrics["trailing_pe"] > 20:
        verdict.append("Expensive valuation"); score -= 10

    # Cash and dividend
    if metrics["cfps"] and metrics["eps"] and metrics["cfps"] > metrics["eps"]:
        verdict.append("Strong cash flow"); score += 5
    if metrics["cash_per_share"] and metrics["current_price"] and metrics["cash_per_share"] > metrics["current_price"] * 0.4:
        verdict.append("High cash reserves"); score += 5
    if metrics["div_yield"] and metrics["div_yield"] > 0.05:
        verdict.append("High dividend yield"); score += 5
    elif metrics["div_yield"] and metrics["div_yield"] < 0.02:
        verdict.append("Low dividend"); score -= 5

    # Liquidity & debt
    if metrics["curr_ratio"] and metrics["curr_ratio"] < 1:
        verdict.append("Weak liquidity"); score -= 5
    elif metrics["curr_ratio"] and metrics["curr_ratio"] > 1.5:
        verdict.append("Healthy liquidity"); score += 5
    if metrics["debt_eq"] and metrics["debt_eq"] > 70:
        verdict.append("High debt load"); score -= 5

    # Profitability
    if metrics["roe"] and metrics["roe"] > 0.15:
        verdict.append("High efficiency (ROE)"); score += 5
    if metrics["roa"] and metrics["roa"] < 0.03:
        verdict.append("Low ROA"); score -= 5
    if metrics["profit_margin"] and metrics["profit_margin"] < 0.05:
        verdict.append("Thin margins"); score -= 5

    # Growth factors
    if metrics["rev_growth"] and metrics["rev_growth"] > 0.15:
        verdict.append("Strong revenue growth"); score += 5
    if metrics["earnings_growth"] and metrics["earnings_growth"] > 0.3:
        verdict.append("Explosive earnings growth"); score += 5

    # Bound the score
    score = max(0, min(100, score))

    # Profile classification
    profile = "Balanced"
    if "Cheap valuation" in verdict and ("High dividend yield" in verdict or "Strong cash flow" in verdict):
        profile = "Cash Cow / Value"
    elif "Expensive valuation" in verdict and ("Strong revenue growth" in verdict or "Explosive earnings growth" in verdict):
        profile = "Growth Bet"
    if "Weak liquidity" in verdict or "High debt load" in verdict or "Thin margins" in verdict:
        profile = "High Risk"

    return {"score": score, "verdict": verdict, "profile": profile}


In [62]:
for item in c_list:
    dat = yf.Ticker(item)
    report = score_stock(dat.info)
    if report.get('score') >= 70:
        print(item)

5TP.SI
AP4.SI
BEZ.SI
ER0.SI
S35.SI
S41.SI
S61.SI
T14.SI
TQ5.SI
U96.SI


In [ ]:
['5GD.SI',
 '5TP.SI',
 'A04.SI',
 'AP4.SI',
 'BEZ.SI',
 'CYW.SI',
 'DHLU.SI',
 'ER0.SI',
 'H02.SI',
 'HMN.SI',
 'J85.SI',
 'S35.SI',
 'S41.SI',
 'S61.SI',
 'T14.SI',
 'TQ5.SI',
 'U96.SI',
 'Z74.SI',
 'Z77.SI',
 'ZKX.SI']

In [109]:
dat = yf.Ticker("KGC")
report = score_stock(dat.info)

In [110]:
report

{'score': 80,
 'verdict': ['Strong cash flow',
  'High dividend yield',
  'Healthy liquidity',
  'High efficiency (ROE)',
  'Strong revenue growth',
  'Explosive earnings growth'],
 'profile': 'Balanced'}

In [108]:
dat.info

{'address1': '1111 Melville Street',
 'address2': 'Suite 820',
 'city': 'Vancouver',
 'state': 'BC',
 'zip': 'V6E 3V6',
 'country': 'Canada',
 'phone': '604 484 4085',
 'website': 'https://www.fortunamining.com',
 'industry': 'Gold',
 'industryKey': 'gold',
 'industryDisp': 'Gold',
 'sector': 'Basic Materials',
 'sectorKey': 'basic-materials',
 'sectorDisp': 'Basic Materials',
 'longBusinessSummary': "Fortuna Mining Corp. engages in the precious and base metal mining and related activities in Argentina, Burkina Faso, Côte d'Ivoire, Mexico, Peru, and Senegal. The company operates through Mansfield, Sanu, Sango, Cuzcatlan, and Bateas segments. It operates the Lindero gold mine, the Yaramoko gold mine, the Séguéla gold mine, and the San Jose silver-gold mine, as well as the Caylloma silver, lead, and zinc mine. The company was formerly known as Fortuna Silver Mines Inc. and changed its name to Fortuna Mining Corp. in June 2024. Fortuna Mining Corp. was incorporated in 1990 and is based in

In [178]:
print(close_series, open_series)

Date
2025-08-27    0.31
2025-08-28    0.31
2025-08-29    0.32
2025-09-01    0.31
2025-09-02     NaN
Name: ZXY.SI, dtype: float64 Date
2025-08-27    0.305
2025-08-28    0.310
2025-08-29    0.310
2025-09-01    0.320
2025-09-02      NaN
Name: ZXY.SI, dtype: float64


In [142]:
c_list

['544.SI',
 '5GD.SI',
 '5TP.SI',
 'AP4.SI',
 'BEZ.SI',
 'DHLU.SI',
 'ER0.SI',
 'H02.SI',
 'KUO.SI',
 'S35.SI',
 'S41.SI',
 'T14.SI',
 'TQ5.SI',
 'Z74.SI',
 'ZKX.SI']

Remove 
OXMU

In [124]:
c_list = []

for item in cols:

    if item in data.T["Open"].columns and item in data.T["Close"].columns:
        close_series = data.T["Close"][item]
        open_series = data.T["Open"][item]
        # print(clo se_series, open_series)
        # do your logic here
    else:
        print(f"Skipping {item} (missing Open/Close)")

    diff = close_series - open_series

    last3 = diff.tail(3)
    check_diff = (last3 > 0).all()

    last4_close = close_series.tail(4)  # need 4 values to compare 3 consecutive increases
    check_close_up = (last4_close.diff().tail(3) >= 0).all()

    if check_diff and check_close_up:
        c_list.append(item)

    

    # if check_diff and check_close_up:

    #     pct_change_open_close = ((close_series - open_series) / open_series) * 100
    #     # print(pct_change_open_close)
    #     close_close_pct = close_series.pct_change() * 100
    #     check = check_trend(close_close_pct)
    
    #     if check:
    #         c_list.append(item)

In [125]:
c_list

['1E3.SI',
 'AWZ.SI',
 'BEZ.SI',
 'F34.SI',
 'H78.SI',
 'J69U.SI',
 'K71U.SI',
 'KUO.SI',
 'ME8U.SI',
 'N02.SI',
 'N2IU.SI',
 'OXMU.SI',
 'Q5T.SI',
 'S35.SI',
 'S41.SI',
 'T82U.SI',
 'X5N.SI',
 'YK9.SI',
 'Z74.SI',
 'ZKX.SI']

In [126]:
c_list = []
c1_list =[]
c2_list = []
c3_list = []

for item in cols:
    # print(item)
    # close_series = data["Close"][item]
    # open_series = data["Open"][item]

    if item in data.T["Open"].columns and item in data.T["Close"].columns:
        close_series = data["Close"][item]
        open_series = data["Open"][item]
        # do your logic here
    else:
        print(f"Skipping {item} (missing Open/Close)")

    diff = close_series - open_series

    last3 = diff.tail(3)
    check_diff = (last3 > 0).all()

    last4_close = close_series.tail(4)  # need 4 values to compare 3 consecutive increases
    check_close_up = (last4_close.diff().tail(3) > 0).all()

    if check_diff and check_close_up:

        pct_change_open_close = ((close_series - open_series) / open_series) * 100
        # print(pct_change_open_close)
        close_close_pct = close_series.pct_change() * 100
        c1, c2 = check_trend(close_close_pct)
    
        if c1 or c2:
            c_list.append(item)
            if c1:
                c1_list.append(item)
            if c2: 
                c2_list.append(item)
        if c1 and c2:
            c3_list.append(item)

KeyError: 'Close'

In [ ]:
response = yf.download(c_list, period='20d')
df2 = response

In [ ]:
final_list = []

for item in c_list:
    print(item)
    dat = yf.Ticker(item)
    # print(dat)
    if dat.info.get("marketCap") and dat.info.get("marketCap") > 100000000:
        vol_series = df2["Volume"][item]
        avg_3 = vol_series.tail(3).mean()
        avg_20 = vol_series.tail(20).mean()
        check = avg_3 > avg_20
        if check:
            final_list.append(item)
        # print(check)
    

In [ ]:
final_list

In [179]:
filtered_df = data[data[today] > 5]

KeyError: '2025-08-27'

In [ ]:
dat = yf.download(['AAPL', 'GOOG', "MSFT"], period='5d')

print(dat)

In [ ]:
close_df = dat["Close"]["AAPL"]

In [ ]:
close_df

In [ ]:
open_df = dat["Open"]["AAPL"]

In [ ]:
open_df

In [ ]:
print(dat.columns)

In [47]:
open_series = dat["Open"]["AAPL"]
close_series = dat["Close"]["AAPL"]

pct_change_open_close = ((close_series - open_series) / open_series) * 100
print(pct_change_open_close)

Date
2025-08-25    0.300251
2025-08-26    1.075507
2025-08-27    0.822363
2025-08-28    0.753830
2025-08-29   -0.159131
Name: AAPL, dtype: float64


In [48]:
def check_trend(pct_series: pd.Series):
    # condition 1: 2 consecutive >= +1%
    cond1 = ((pct_series >= 1).astype(int)
             .rolling(2).sum() >= 2).any()

    # condition 2: 3 consecutive > 0
    cond2 = ((pct_series > 0).astype(int)
             .rolling(3).sum() >= 3).any()

    return cond1, cond2

In [49]:
pct_change_open_close = ((close_series - open_series) / open_series) * 100
c1, c2 = check_trend(pct_change_open_close)

print("2 consecutive >= +1%:", c1)
print("3 consecutive > 0:", c2)

2 consecutive >= +1%: False
3 consecutive > 0: True
